In [1]:
def auth():
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    return(drive)

def read(drive):
    file = drive.ListFile({'q': "title = 'TEN ENG' "}).GetList()[0]
    file.GetContentFile(file['title']+'.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    df = pd.read_excel('TEN ENG.xlsx')
    dff = df.groupby('ma_email').last()
    
    d_email_name = dict(dff['ma_name'])
    d_nameemail_email = {name+' ({})'.format(email): email for email, name in d_email_name.items()}
    names = list(d_nameemail_email.keys())
    return(dff, d_nameemail_email, names)

def read_1(drive):
    file = drive.ListFile({'q': "title = 'TENENG_VOL2' "}).GetList()[0]
    file.GetContentFile(file['title']+'.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    df = pd.read_excel('TENENG_VOL2.xlsx')
    dff = df.groupby('ma_email').last()
    
    d_email_name = dict(dff['ma_name'])
    d_nameemail_email = {name+' ({})'.format(email): email for email, name in d_email_name.items()}
    names = list(d_nameemail_email.keys())
    return(dff, d_nameemail_email, names)

def make_window(names):
    layout = [
        [sg.HorizontalSeparator()],
        [sg.Radio('Кабинет 1 (вопросы 1-14)', 'Type', key='Radio', default=True,  enable_events=True)],
        [sg.Radio('Кабинет 2 (вопросы 14-20)', 'Type', key='Radio_1', enable_events=True)],
        [sg.Combo(names, key='Name', readonly=True, size=(46, 10))],
        [sg.Submit('Запустить'), sg.Cancel('Выйти')],
        [sg.HorizontalSeparator()],
        ]
    return sg.Window('Результаты кабинета. Тен Инжиниринг', layout, size=(390,150))

def make_window1(file_name):
    layout1 = [
        [sg.Text(f"Файл не найден: {file_name}")],
        [sg.Button('Понятно')]
        ]
    return sg.Window('Ой', layout1).read(close=True)

def make_window2(file_name):
    layout1 = [
        [sg.Text(f"Файл создан: {file_name}")],
        [sg.Button('Закрыть')]
        ]
    return sg.Window('Готово', layout1).read(close=True)

def make_window3(error_text):
    layout1 = [
        [sg.Text(error_text)],
        [sg.Button('Понятно')]
        ]
    return sg.Window('Ой', layout1).read(close=True)

In [8]:
import PySimpleGUI as sg
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pandas as pd
from docxtpl import DocxTemplate
import warnings
warnings.filterwarnings('ignore')

def main():
    sg.theme('Reddit')
    drive = auth()
    
    try:
        dff, d_names, names = read(drive)
    except Exception as e:
        print(e)
        names, window1 = [], make_window1('TEN ENG')
        return
    try:
        dff_1, d_names_1, names_1 = read_1(drive)
    except:
        names, window1 = [], make_window1('TENENG_VOL2')
        return
        """
    try:
        doc = DocxTemplate('Шаблон_ответа_Ten_eng.docx')
        doc_1 = DocxTemplate('Шаблон_ответа_Ten_eng_vol_2.docx')
    except:
        window1 = make_window1('Шаблон_ответа_Ten_eng.docx')
    try:
        doc_1 = DocxTemplate('Шаблон_ответа_Ten_eng_vol_2.docx')
    except:
        window1 = make_window1('Шаблон_ответа_Ten_eng_vol_2.docx')
        """
    window = make_window(names)
    doc = DocxTemplate('Шаблон_ответа_Ten_eng.docx')
    doc_1 = DocxTemplate('Шаблон_ответа_Ten_eng_vol_2.docx')
    
    while True:
        event, values = window.read()
        if event in (None, 'Exit', 'Выйти'):
            break
        if event == 'Radio':
            window['Name'].update(values=names)
        if event == 'Radio_1':
            window['Name'].update(values=names_1)
        if event == 'Запустить':
            try:
                if values['Radio']:
                    email = d_names[values['Name']]
                    context = {'ma_email': email}
                    context.update(dict(dff.loc[email]))
                    filename = values['Name']+'.Каб1.docx'
                    doc.render(context)
                    doc.save(filename)
                    window2 = make_window2(filename)
                else:
                    email = d_names_1[values['Name']]
                    context = {'ma_email': email}
                    context.update(dict(dff_1.loc[email]))
                    filename = values['Name']+'.Каб2.docx'
                    doc_1.render(context)
                    doc_1.save(filename)
                    window2 = make_window2(filename)
            except Exception as e:
                window3 = make_window3(e)
                pass
    window.close()
    
if __name__ == "__main__":
    main()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=447909346377-de10dblc2ocggjqak276nselomb6s6uf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
